<a href="https://colab.research.google.com/github/fetterollie/rec-systems-project/blob/main/web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creat web app on Gradio

## Install, import and mount

In [1]:
# Install Gradio
!pip install gradio

# Import necessary libraries
import gradio as gr
import pandas as pd

# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Get the saved dataframe for userid = 2

In [2]:
df = pd.read_csv('/content/gdrive/MyDrive/rec-systems-project/data/user2_prediction_result.csv')
df.head(10)

,Unnamed: 0,title,prediction,genres,split_genres
0,899,Star Wars: Episode V - The Empire Strikes Back...,4.411833,action|adventure|sci-fi,"['action', 'adventure', 'sci-fi']"
1,2229,Fight Club (1999),4.386436,action|crime|drama|thriller,"['action', 'crime', 'drama', 'thriller']"
2,600,Dr. Strangelove or: How I Learned to Stop Worr...,4.383926,comedy|war,"['comedy', 'war']"
3,907,Lawrence of Arabia (1962),4.363100,adventure|drama|war,"['adventure', 'drama', 'war']"
4,660,"Godfather, The (1972)",4.349665,crime|drama,"['crime', 'drama']"
5,695,Casablanca (1942),4.341353,drama|romance,"['drama', 'romance']"
6,900,"Princess Bride, The (1987)",4.324890,action|adventure|comedy|fantasy|romance,"['action', 'adventure', 'comedy', 'fantasy', '..."
7,46,"Usual Suspects, The (1995)",4.321951,crime|mystery|thriller,"['crime', 'mystery', 'thriller']"
8,910,Apocalypse Now (1979),4.315185,action|drama|war,"['action', 'drama', 'war']"
9,224,Star Wars: Episode IV - A New Hope (1977),4.313092,action|adventure|sci-fi,"['action', 'adventure', 'sci-fi']"


## Data cleaning: round 'prediction' column

In [3]:
df['prediction'] = round(df['prediction'], 2)

## Data cleaning: clean "split_genres" column and collect the genre list

Need the genre list as the genre options for web app

In [4]:
# Clean "split_genres" column

test_df = df.copy()
test_df['split_genres']=test_df['split_genres'].str[1:-1].str.split(',').tolist()
test_df.head()

,Unnamed: 0,title,prediction,genres,split_genres
0,899,Star Wars: Episode V - The Empire Strikes Back...,4.41,action|adventure|sci-fi,"['action', 'adventure', 'sci-fi']"
1,2229,Fight Club (1999),4.39,action|crime|drama|thriller,"['action', 'crime', 'drama', 'thriller']"
2,600,Dr. Strangelove or: How I Learned to Stop Worr...,4.38,comedy|war,"['comedy', 'war']"
3,907,Lawrence of Arabia (1962),4.36,adventure|drama|war,"['adventure', 'drama', 'war']"
4,660,"Godfather, The (1972)",4.35,crime|drama,"['crime', 'drama']"


In [5]:
# Check the genre_df

genre_df = test_df['split_genres']
# genre_df.head(10)

In [6]:
# Collect genre_list and more cleaning

genre_list=[]

for each in genre_df:
  for i in each:
    if i not in genre_list:
      genre_list.append(i.title())

genre_list = [i.strip("'").replace(" ", "").strip("'") if type(i) == str else str(i) for i in genre_list]

In [7]:
# Check genre_list

genre_list

['Action',
 'Adventure',
 'Sci-Fi',
 'Action',
 'Crime',
 'Drama',
 'Thriller',
 'Comedy',
 'War',
 'Adventure',
 'Drama',
 'War',
 'Crime',
 'Drama',
 'Drama',
 'Romance',
 'Action',
 'Adventure',
 'Comedy',
 'Fantasy',
 'Romance',
 'Crime',
 'Mystery',
 'Thriller',
 'Action',
 'Drama',
 'War',
 'Action',
 'Adventure',
 'Sci-Fi',
 'Mystery',
 'Thriller',
 'Mystery',
 'Thriller',
 'Action',
 'Adventure',
 'Drama',
 'Adventure',
 'Comedy',
 'Fantasy',
 'Drama',
 'Drama',
 'War',
 'Crime',
 'Mystery',
 'Thriller',
 'Action',
 'Sci-Fi',
 'Thriller',
 'Crime',
 'Drama',
 'Crime',
 'Drama',
 'Crime',
 'Drama',
 'Comedy',
 'Crime',
 'Thriller',
 'Drama',
 'Romance',
 'Sci-Fi',
 'Action',
 'Sci-Fi',
 'Thriller',
 'Action',
 'Adventure',
 'Drama',
 'Fantasy',
 'Crime',
 'Horror',
 'Thriller',
 'Comedy',
 'Drama',
 'Romance',
 'War',
 'Drama',
 'Comedy',
 'Crime',
 'Drama',
 'Thriller',
 'Horror',
 'Action',
 'Adventure',
 'Drama',
 'War',
 'Comedy',
 'Crime',
 'Drama',
 'Thriller',
 'Drama',
 

In [8]:
# Remove duplicate and sort
genre_list = sorted(list(set(genre_list)))

# Remove '(nogenreslisted)'
genre_list.remove('(NoGenresListed)')

In [9]:
# Check genre_list again

genre_list

['Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Imax',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

# Deploy web app 1 - output movie list

Don't like this one. Too simple and not pretty.

In [10]:

# def movie_recommendation(rating, genre, number):
#   """
#   number = how many recommended movies
#   rating = select movie rating
#   genre = select movie genre
#   """
    
#   # number = 5, 10, 15, 20
#   # rating = 3.35
#   # genre = 'action'
  
#   filter_rating_df = df[(df['prediction'] <= rating+0.5) &
#                      (df['prediction'] >= rating-0.5)]
#   filter_df = filter_rating_df[filter_rating_df['split_genres'].str.contains(genre, case=False)]
#   rec_movie = filter_df['title'][:int(number)].tolist()
#   return rec_movie

# rating = gr.inputs.Slider(1, 5, label='Slect movie ratings')
# genre = gr.inputs.Dropdown(genre_list, label='Pick a movie genre')

# number = gr.Number(label='How many recommended movies?')
# # number = gr.inputs.CheckboxGroup([5,10,15,20], type="index", label='How many recommended movies?')

# gr.Interface(fn=movie_recommendation,
#              inputs=[rating, genre, number],
#              outputs=['text'],
#              title= "Movie recommendations for you").launch(debug=True, share=True)

# Deploy web app 2 - output title and predicted rating dataframe

Like the output as table but still need some improvement.

In [11]:
# def movie_recommendation(rating, genre, number):
#   """
#   number = how many recommended movies
#   rating = select movie rating higher
#   genre = select movie genre
#   """
    
#   # number = Int (ex. 5)
#   # rating = Float (ex. 3.5)
#   # genre = Str (ex. 'action')
  
#   filter_rating_df = df[df['prediction'] >= rating]
#   filter_df = filter_rating_df[filter_rating_df['split_genres'].str.contains(genre, case=False)]
#   rec_movie = filter_df[['title', 'prediction']][:int(number)]
#   rec_movie.rename(columns={'prediction':'Predicted Rating', 'title': 'Title'}, inplace=True)
#   return rec_movie

# # Gradio
# rating = gr.inputs.Slider(1, 5, label='Only want movies with ratings higher than:')
# genre = gr.inputs.Dropdown(genre_list, label='Pick a movie genre:')
# number = gr.Number(label='How many recommended movies?')
# output = gr.Dataframe(headers=["title", "predicted rating"])


# gr.Interface(fn= movie_recommendation,
#              inputs=[rating, genre, number],
#              outputs=output,
#              title= "Movie recommendations for [UserName]").launch(debug=True, share=True)


# Deploy web app 3 - change choice of movie list to checkboxgoup

Don't like this one, it allows checked multiple boxes.

In [12]:
# from collections import namedtuple 

# def movie_recommendation(rating, genre, choice):

#   filter_rating_df = df[df['prediction'] >= rating]
#   filter_df = filter_rating_df[filter_rating_df['split_genres'].str.contains(genre, case=False)]
#   rec_movie = filter_df[['title', 'prediction']]
#   rec_movie.rename(columns={'prediction':'Predicted Rating', 'title': 'Title'}, inplace=True)
    
#   def top_slice(x, y):
#     slice_df= rec_movie[x:y]
#     return slice_df
   
#   top_5 = top_slice(0, 5)
#   top_10 = top_slice(0, 10)
#   top_20 = top_slice(0, 20)
#   second_10 = top_slice(10, 20)
#   bottom_5 = top_slice(-5, None)
#   bottom_10 = top_slice(-10, None)
  
#   choice_dict = {'Top_5':top_5, 'Top_10':top_10, 'Top_20':top_20, 'Second_10': second_10, 'Bottom_5': bottom_5, 'Bottom_10':bottom_10}
  
#   temp = namedtuple("temp", "choice_str")
#   res = temp(*choice)

#   return choice_dict[str(res.choice_str)]

# choice_list = ['Top_5', 'Top_10', 'Top_20', 'Second_10', 'Bottom_5', 'Bottom_10']

# # Gradio
# rating = gr.inputs.Slider(1, 5, label='Movies with ratings higher than:')
# genre = gr.inputs.Dropdown(genre_list, label='Pick a movie genre:')
# choice = gr.inputs.CheckboxGroup(choice_list, label='Choose the recommended movies list:')
# output = gr.Dataframe(headers=["Title", "Predicted Rating"])


# gr.Interface(fn= movie_recommendation,
#              inputs=[rating, genre, choice],
#              outputs=output,
#              title= "Movie Recommendations").launch(debug=True, share=True)

# Deploy web app 4 - change choice of movie list to radio buttom

## Our final choice!!

In [13]:
def movie_recommendation(rating, genre, choice):

  filter_rating_df = df[df['prediction'] >= rating]
  filter_df = filter_rating_df[filter_rating_df['split_genres'].str.contains(genre, case=False)]
  rec_movie = filter_df[['title', 'prediction']]
  rec_movie.rename(columns={'prediction':'Predicted Rating', 'title': 'Title'}, inplace=True)
    
  def top_slice(x, y):
    slice_df= rec_movie[x:y]
    return slice_df
   
  top_5 = top_slice(0, 5)
  top_10 = top_slice(0, 10)
  top_20 = top_slice(0, 20)
  second_10 = top_slice(10, 20)
  bottom_5 = top_slice(-5, None)
  bottom_10 = top_slice(-10, None)
  
  choice_dict = {'Top_5':top_5, 'Top_10':top_10, 'Top_20':top_20, 'Second_10': second_10, 'Bottom_5': bottom_5, 'Bottom_10':bottom_10}

  return choice_dict[choice]

choice_list = ['Top_5', 'Top_10', 'Top_20', 'Second_10', 'Bottom_5', 'Bottom_10']

# Gradio
rating = gr.inputs.Slider(1, 5, label='Movies with ratings higher than:')
genre = gr.inputs.Dropdown(genre_list, label='Pick a movie genre:')
choice = gr.inputs.Radio(choice_list, label='Choose the recommended movies list:')
output = gr.Dataframe(headers=["Title", "Predicted Rating"])


gr.Interface(fn= movie_recommendation,
             inputs=[rating, genre, choice],
             outputs=output,
             title= "Movie Recommendations").launch(debug=True, share=True)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:183: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a8212517-d189-477b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a8212517-d189-477b.gradio.live
